#### ZJ_Zhang_W12_assn.py

### Part I: Working with HTML, XML, and JSON

* pick three books
* create the HTML, XML, JSON files separately to store books' information
* load three files into separate Pandas Data Frames

In [14]:
import numpy as np
import pandas as pd

#### The books i choosed are about the Thanksgiving day for children


### create json file

In [6]:
import json
# pd.read_json returns a data frame
data1 = pd.read_json('https://raw.githubusercontent.com/zhijing-zhang/DAV-5400/master/books.json')
data1

,Title,Author,Age Range,Publisher,Amazon best rank
0,Thanksgiving in the Woods,"[Phyllis Alsdurf(Author), Jenny Lovlie(Illustr...",4 - 7,Sparkhouse Family (October 17 2017),38
1,Thanksgiving Is for Giving Thanks!,"[Margaret Sutherland, Sonja Lamut (Author)]",3 - 5,Grosset & Dunlap (September 11 2000),11
2,Turkey Trouble,"[Wendi Silvano(Author), Lee Harper(Illustrator)]",3 - 7,Two Lions; 8th Print edition (October 1 2009),2


### create HTML file

In [7]:
# pd.read_html returns a Python list object
data2 = pd.read_html('https://raw.githubusercontent.com/zhijing-zhang/DAV-5400/master/books.html')
# the first item (and in this case, the only item) in the list is a data frame
f = data2[0]
f

,Title,Author,Age Range,Publisher,Amazon Best Rank
0,Thanksgiving in the Woods,"Phyllis Alsdurf(Author),Jenny Lovlie(Illustrator)",4-7,"Sparkhouse Family (October 17, 2017)",38
1,Thanksgiving Is for Giving Thanks!,"Margaret Sutherland, Sonja Lamut (Author)",3-5,"Grosset & Dunlap (September 11, 2000)",11
2,Turkey Trouble,"Wendi Silvano(Author), Lee Harper(Illustrator)",3-7,"Two Lions; 8th Print edition (October 1, 2009)",2


### create XML fiel

* capture the path and header from XML file
* parse the page
* get the root node of XML file('PERFORMANCE' in this case)
* skip some fields that we don't need ('PARENT_SEQ', 'INDICATOR_SEQ' in this case)
* find the infromation in each 'INDICATOR' and ues them to create a dataframe

In [8]:
# load the urllib.request function so that we can use a web path with the 
# Python open() function
import urllib.request

# load the objectify() function from the lxml library
from lxml import objectify

# open the web page containing the data set: 
# be sure to capture the header info separately from the path
path, headers = urllib.request.urlretrieve('https://raw.githubusercontent.com/zhijing-zhang/DAV-5400/master/books.xml')

# objectify() is then used to parse the web page
parsed = objectify.parse(open(path))
#parsed

# now get a reference to the root node of the XML file
# in this case, root is the 'PERFORMANCE'
root = parsed.getroot()

# define an empty list that will be used to store the parsed data
data = []

# define a list of XML fields that we will ignore for this particular data set: 
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ']

# root.INDICATOR is a generator that we use to extract each <INDICATOR> element from the XML data
for elt in root.INDICATOR:
     # for each record, create a dict of tag names
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)
    
# now check the results
perf = pd.DataFrame(data)
perf

,Title,Author,Age_Range,Publisher,Amazon_Best_rank
0,Thanksgiving in the Woods,"Phyllis Alsdurf(Author),Jenny Lovlie(Illustrator)",4-7,"Sparkhouse Family (October 17, 2017)",38
1,Thanksgiving Is for Giving Thanks!,"Margaret Sutherland, Sonja Lamut (Author)",3-5,"Grosset & Dunlap (September 11, 2000)",11
2,Turkey Trouble,"Wendi Silvano(Author), Lee Harper(Illustrator)",3-7,"Two Lions; 8th Print edition (October 1, 2009)",2


#### Conclusion:

* The json file can load the information to data frame directely
* The HTML file needs one more step to transfer to data, from list to data frame
* The XML file has the most complicated format, so it can't load the information into a dataframe directely, it needs more steps like above
* I think the final three data frames are totally same

### Part II: Working with Web API’s

* choose one of the New York Times APIs  
* construct an interface in Python to read JSON data accessible via the API 
* transform that data into a Pandas data frame

#### The API i choosed is Times Newswire API which is about real-time feed of NYC artical publishes

* API key: VwAldpzqaxdWpZGDeYwuonEoHOFO9Vfy
* data can be accessed via JSON format in NY times

In [9]:
import requests

url = 'https://api.nytimes.com/svc/news/v3/content/all/all.json?api-key=VwAldpzqaxdWpZGDeYwuonEoHOFO9Vfy'
resp = requests.get(url)
resp

<Response [200]>

In [13]:
# now convert the 'resp' object's JSON content into a list of native Python objects
# in this case, first we can get a dict
data = resp.json()
type(data)

# from the dict{copyright, number_results, results, status}, choose the list "results" as our information
data1 = data['results']
data1

# view the content of that dict object
data1[0]

{'slug_name': '30wordplay-mood',
 'section': 'Crosswords & Games',
 'subsection': '',
 'title': 'Mood',
 'abstract': 'Find somewhere quiet and settle down with this Joe Deeney puzzle.',
 'url': 'https://www.nytimes.com/2019/11/29/crosswords/daily-puzzle-2019-11-30.html',
 'byline': 'BY CAITLIN LOVINGER',
 'thumbnail_standard': 'https://static01.nyt.com/images/2019/12/01/crosswords/01wordplay-hermit/01wordplay-hermit-thumbStandard.jpg',
 'item_type': 'Article',
 'source': 'New York Times',
 'updated_date': '2019-11-29T21:40:02-05:00',
 'created_date': '2019-11-29T21:40:02-05:00',
 'published_date': '2019-11-29T21:40:02-05:00',
 'first_published_date': '2019-11-29T21:40:02-05:00',
 'material_type_facet': 'News',
 'kicker': 'wordplay, the CROSSWORD COLUMN',
 'subheadline': '',
 'des_facet': ['Crossword Puzzles', 'Poetry and Poets'],
 'org_facet': None,
 'per_facet': ['Deeney, Joe (Crossword Constructor)'],
 'geo_facet': None,
 'related_urls': None,
 'multimedia': [{'url': 'https://static0

In [10]:
# now convert some of the extracted JSON data into a data frame: 
# in this example we are extracting the 'slug_name', 'section', 'subsection', 'title', 'abstract' components of the dict object
issues = pd.DataFrame(data1, columns=['slug_name', 'section',
                                     'subsection', 'title', 'abstract'])
issues

,slug_name,section,subsection,title,abstract
0,30wordplay-mood,Crosswords & Games,,Mood,Find somewhere quiet and settle down with this...
1,29siwoff,Sports,,"Seymour Siwoff, Master of Sports Statistics, I...","As head of the Elias Sports Bureau, which he t..."
2,30weekend-reads,U.S.,,11 of Our Best Weekend Reads,"An inside look at turkey hunters, the changing..."
3,29nba-nets,Sports,Pro Basketball,The Nets Win One for Their Culture,"After beating the Celtics on Friday, the Nets ..."
4,29canadaletter,World,Canada,"Shmoo Cake, Persians and Spudnuts: Touring Can...",In appreciation of the delicacies that provoke...
5,29suriname,World,Americas,Suriname President Convicted in 1982 Killings,The decision by a panel of three judges marked...
6,29nba-signlanguage,Sports,,Giannis Antetokounmpo Is Called Amazing. Now i...,New this season at the Milwaukee Bucks’ postga...
7,29xp-teacher,U.S.,,Teacher Fired After Asking Trump to ‘Remove’ I...,A Texas state agency ruled in favor of the tea...
8,29Bolivia-ES,en Español,América Latina,‘Evo Morales es como un padre para nosotros’,En la región cocalera donde el expresidente de...
9,29nfl,Sports,Pro Football,N.F.L. Suspends Cardinals Player for Betting,Josh Shaw has spent the season on the injured ...
